In [ ]:
using Pkg
Pkg.activate("../..")
using Revise
using TGLFNN
using FUSE
using Plots;
FUSE.logging(Logging.Info; actors=Logging.Info);

In [ ]:
# sty is the act equivalent for a study, it has common parameters like server and n_workers but also study dependent parameters like n_simulations
sty,act = FUSE.study_parameters(:DatabaseGenerator);
sty

In [ ]:
using IMAS
dd = IMAS.json2imas("/mnt/beegfs/users/neisert/ODSs/d3d/fuse_prepared_inputs/ods_161582_2300.json");

In [ ]:
plot(dd.equilibrium,wall)

In [ ]:
# Interacting with sty
sty.server = "saga"
sty.n_workers = 2

sty.save_folder = "/mnt/beegfs/users/neisert/ODSs/ukaea/seq-2"

sty.release_workers_after_run = true # this is the default behavior and releases workers after running the study

sty.n_simulations = 4

sty

In [ ]:
dd = IMAS.h5i2imas("/mnt/beegfs/users/neisert/ODSs/ukaea/seq-1/imasdb/step/3/88888/2/master.h5");
dd.global_time=102.07396074300034
chk = FUSE.Checkpoint()
chk[:init] = dd, FUSE.ParametersInits(),FUSE.ParametersActors()
#dd.equilibrium.time_slice[].profiles_2d.psi = dd.equilibrium.time_slice[].profiles_2d.psi.T #grid1 grid2
#FUSE.digest(dd)


In [ ]:
ini_step = FUSE.ParametersInits();
ini_step.general.dd = dd
ini_step.general.init_from=:ods

FUSE.ini_from_ods!(ini_step;restore_expressions=false)

ini_step

In [ ]:
dd.global_time = 1500.
plot(dd.equilibrium)

In [ ]:
dd.global_time = 1500.
plot(dd.equilibrium)

In [ ]:
standard_lmode_dcit = FUSE.profiles[:Lmode]
standard_lmode_dcit.tedge = 3.0 ↔ [2.5,3.5]

ini_step.core_profiles.profiles = 

In [ ]:
p = plot(dd.equilibrium)

ini_step.general.dd = missing
ini_step.general.init_from = :scalars

ini_step.core_profiles.greenwald_fraction_ped = 0.5 ↔ [0.3,0.85]
ini_step.core_profiles.T_ratio = 0.5 ↔ [0.15,1.2]

ini_step.equilibrium.boundary_from = :scalars
ini_step.core_profiles.zeff = 3.0 ↔ [2.5,3.5]

# Tom can figure these out
ini_step.core_profiles.w_ped = 0.05 #↔ [2.5,3.5]



p0 =  630797.
ini_step.equilibrium.pressure_core = p0 ↔ [p0,p0*1.5]
ini_step.core_profiles.zeff = 3.0 ↔ [2.5,3.5]

ini_step.core_profiles.T_shaping =  1.5 ↔ [0.8,2.2]
ini_step.core_profiles.n_shaping =  1.5 ↔ [0.8,2.2]
ini_step.core_profiles.rot_core = 0.0 

ini_step.core_profiles.impurity = :Ar
ini_step.core_profiles.bulk = :DT
ini_step.core_profiles.helium_fraction = 0.0
#ini_step.core_profiles.zeff = 3.0 ↔ [2.5,3.5]

for i in 1:3
    @show i
    dd_step = IMAS.dd()
    
    FUSE.rand!(ini_step)
    FUSE.init(dd_step,ini_step,act)
    plot!(p,dd_step.equilibrium)
end
display(p)
# add a ne_sep / ne_ped ratio to the ini and set that accordingly to the s

In [ ]:
p = plot(dd.core_profiles.profiles_1d[].electrons.temperature)
for shape_1 in 0.8:0.1:2.2
    for shape_2 in 0.8:0.1:2.2
        profile = IMAS.Hmode_profiles(
        plot!(p,profile)
    end
end
display(p)


In [ ]:
#ini_step.core_profiles.profiles = Dict(:L_mode => Dict(:core=>1e4,:edge=70,:shape=0.8,.....),:Hmode=> Dict(:bla=>10), :ITB=> Dict())

In [ ]:
# Function to find the index of the nearest value
function find_nearest_index(array, target)
    differences = abs.(array .- target)  # Compute the absolute differences
    nearest_index = argmin(differences)  # Find the index of the smallest difference
    return nearest_index
end
index_of_nearest = find_nearest_index(dd.summary.time, dd.global_time)
dd.summary.local.pedestal.t_e.value[index_of_nearest]

IMAS.Hmode_profiles(dd.core_profiles.profiles_1d[].electrons.temperature[end], dd.summary.local.pedestal.t_e.value[index_of_nearest], dd.core_profiles.profiles_1d[].electrons.temperature[1],length(dd.core_profiles.profiles_1d[].grid.rho_tor_norm))


In [ ]:
length(dd.core_profiles.profiles_1d[].grid.rho_tor_norm)

In [ ]:
plot(dd.core_profiles;only=2)

In [ ]:
plot(dd.core_profiles)

In [ ]:
ini.core_profiles.zeff = 2.5 ↔ [1.4,4.5]
ini.core_profiles.greenwald_fraction_ped = 0.2 ↔[0.2,0.75*0.9]
ini.core_profiles.temp_shape 

In [ ]:
dd,ini,act = chk[:init]
dd.global_time= 14.1
ini_test = FUSE.ParametersInits()
ini_test.general.init_from =:ods
ini_test.general.dd = dd
ini_test
FUSE.ini_from_ods!(ini_test;restore_expressions=false)

In [ ]:
#dd = IMAS.h5i2imas("/mnt/beegfs/users/neisert/ODSs/ukaea/seq-2/master.h5");
dd,ini,act = chk[:init]
rhorange = range(0.1, 0.9, 9)
act = FUSE.ParametersActors()
act.ActorTGLF.rho_transport = rhorange

t = 20
#=dd.global_time=dd.summary.time[t+1]
dd.equilibrium.time_slice[dd.summary.time[t+1]].profiles_2d[1].psi = Matrix(transpose(dd.equilibrium.time_slice[t].profiles_2d[1].psi))
dd.equilibrium.time_slice[dd.summary.time[t+1]].profiles_1d.squareness_lower_inner = zeros(100)
dd.equilibrium.time_slice[dd.summary.time[t+1]].profiles_1d.squareness_lower_outer = zeros(100)
dd.equilibrium.time_slice[dd.summary.time[t+1]].profiles_1d.squareness_upper_inner = zeros(100)
dd.equilibrium.time_slice[dd.summary.time[t+1]].profiles_1d.squareness_upper_outer = zeros(100)

actor_tglf = FUSE.ActorTGLF(dd,act;rho_transport=rhorange) #dd_iter
TGLFNN.save(actor_tglf.input_tglfs[1], "./input_tglfs_seq1/input.tglf_test_2")
=#
#ini_test = FUSE.ParametersInits()
#ini_test.general.init_from =:ods
#ini_test.general.dd = dd
#ini_test
#FUSE.ini_from_ods!(ini_test;restore_expressions=false)

plot(dd.equilibrium.time_slice[t])

In [ ]:
ini_test

In [ ]:
#for (t) in dd.summary.time
#    println(t);
#end
println(size(dd.summary.time))
println(dd.summary.time[1])
println(dd.equilibrium.time_slice[dd.summary.time[1]].time)
println(dd.summary.time[2])
println(dd.equilibrium.time_slice[dd.summary.time[3]].time)
println(dd.summary.time[19])
println(dd.summary.time[20])
println(dd.equilibrium.time_slice[dd.summary.time[20]].time)
println(dd.equilibrium.time_slice[dd.summary.time[21]].time)

In [ ]:
FUSE.ActorTGLF

In [ ]:
rhorange = range(0.1, 0.9, 9)
act = FUSE.ParametersActors()
#act.ActorTGLF.tglfnn_model = "sat1_em_d3d_azf-1"


for t in 1:1:2
    println(dd.summary.time[t])
    println(dd.equilibrium.time_slice[dd.summary.time[t+1]].time)
    time = round(Int,dd.summary.time[t])
    dd.global_time=dd.summary.time[t]
    #=dd.equilibrium.time_slice[dd.summary.time[t+1]].profiles_2d[1].psi = Matrix(transpose(dd.equilibrium.time_slice[t].profiles_2d[1].psi))
    dd.equilibrium.time_slice[dd.summary.time[t+1]].profiles_1d.squareness_lower_inner = zeros(100)
    dd.equilibrium.time_slice[dd.summary.time[t+1]].profiles_1d.squareness_lower_outer = zeros(100)
    dd.equilibrium.time_slice[dd.summary.time[t+1]].profiles_1d.squareness_upper_inner = zeros(100)
    dd.equilibrium.time_slice[dd.summary.time[t+1]].profiles_1d.squareness_upper_outer = zeros(100)=#
    #dd.equilibrium.time_slice[dd.summary.time[t]].profiles_2d[1].psi = Matrix(transpose(dd.equilibrium.time_slice[dd.summary.time[t]].profiles_2d[1].psi))
    dd.equilibrium.time_slice[dd.summary.time[t]].profiles_1d.squareness_lower_inner = zeros(100)
    dd.equilibrium.time_slice[dd.summary.time[t]].profiles_1d.squareness_lower_outer = zeros(100)
    dd.equilibrium.time_slice[dd.summary.time[t]].profiles_1d.squareness_upper_inner = zeros(100)
    dd.equilibrium.time_slice[dd.summary.time[t]].profiles_1d.squareness_upper_outer = zeros(100)
    
    actor_tglf = FUSE.ActorTGLF(dd,act;rho_transport=rhorange, electromagnetic=true, lump_ions=false) #dd_iter
     for k in 1:length(rhorange)
         println(k, " ", rhorange[k])
         TGLFNN.save(actor_tglf.input_tglfs[k], "./input_tglfs_seq1_v2/input.tglf_$(time)_$(rhorange[k])")
     end
    plot(dd.equilibrium.time_slice[t])
end

In [ ]:
dd.core_profiles.profiles_1d[t].time

In [ ]:
dd.equilibrium.time_slice[t].profiles_1d

In [ ]:
plot(dd.equilibrium.time_slice[t].profiles_1d.psi)

In [ ]:
dd.equilibrium.time_slice[30]

In [ ]:
dd.global_time = 2000.
FUSE.digest(dd)

In [ ]:
dd_init = deepcopy(dd);

In [ ]:
dd = deepcopy(dd_init)
act= FUSE.ParametersActors();

dd.global_time = 1600.
act.ActorTGLF.model= :TGLFNN
act.ActorTGLF.lump_ions = false
#FUSE.ActorTGLF(dd,act)
#dd.equilibrium.time_slice[].profiles_1d.j_tor = abs.(dd.equilibrium.time_slice[].profiles_1d.j_tor)
actor_teq = FUSE.ActorTEQUILA(dd,act;ip_from=:equilibrium)
#plot(dd.equilibrium)

In [ ]:
dd_init.equilibrium.time_slice[].profiles_2d

In [ ]:
ini,act = FUSE.case_parameters(:STEP);

ini_new = FUSE.ParametersInits()

ini_new.general.init_from=  :ods
ini_new.time.simulation_start = 1500.
FUSE.ini_from_ods!(ini;restore_expressions=false)
#dd.global_time = X
#dd_init.global_time = 1500.

ini_new.general.dd = deepcopy(dd_init);

dd = IMAS.dd()
ini_new.general.init_from=  :scalars

FUSE.init(dd,ini_new,act);

In [ ]:
dd

In [ ]:
plot(dd.equilibrium.time_slice[1500.])

In [ ]:
# setup ini
ini, act = FUSE.case_parameters(:ITER;init_from=:scalars)

# core profiles
ini.core_profiles.zeff = 2.5 ↔ [1.4,4.5]
ini.core_profiles.greenwald_fraction_ped = 0.2 ↔[0.2,0.75*0.9]
ini.core_profiles.temp_shape 
# H&CD
ini.nb_unit[1].power_launched = 33.4e6 ↔ [0.0,33.4e6]
ini.nb_unit[1].beam_energy = 1e6 ↔ [1e5,1e6]
ini.ec_launcher[1].power_launched = 20e6 ↔ [0.0,20e6]
ini.ec_launcher[1].rho_0 = 0.3 ↔ [0.1,0.9]
ini.ic_antenna[1].power_launched = 24e6 ↔ [0.0,24e6]
ini.lh_antenna[1].power_launched = 10e6 ↔ [0.0,10e6]

# Change act for running ITER for IDA
act.ActorCoreTransport.model = :FluxMatcher
act.ActorFluxMatcher.evolve_pedestal = false
act.ActorTGLF.warn_nn_train_bounds = false
act.ActorFluxMatcher.evolve_rotation = :fixed
act.ActorFluxMatcher.evolve_densities = :flux_match

In [ ]:
# study is the actor equivalent of actors, here sty, act and outputs are kept of the workflow
study = FUSE.StudyDatabaseGenerator(sty,ini, act; n_workers=2); # it is possible to pass in keyword arguments to sty

# instantiating the study will also setup the study i.e. FUSE.setup(study);

In [ ]:
# for now you will have to import FUSE everywhere in your distributed computing
using Distributed
@everywhere import FUSE


In [ ]:
FUSE.run(study);   # runs the study, this also writes the dataframes

# at the end of the run workers are released so you will have to setup again if you want to run more

In [ ]:
# data for this workflow is stored in a dictionary of dataframes study.dataframes_dict
study.dataframes_dict["outputs_summary"]



In [ ]:
# analyze the study results 
FUSE.analyze(study);


